In [0]:
from google.colab import drive
drive.mount('/gdrive')

In [0]:
path = "/gdrive/My Drive/machathon-10-filteration-test/train.csv"
path2 = "/gdrive/My Drive/machathon-10-filteration-test/test.csv"

In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
import numpy as np
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import NearestNeighbors
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV

warnings.filterwarnings('ignore')



# Loading and exploring data

In [0]:
#bring in the six packs
data_train = pd.read_csv(path)
data_test = pd.read_csv(path2)

In [0]:
data_train.head()

In [0]:
data_train.describe()

# Removing low correation fetures

In [0]:
# correlatio between fetures
corr = data_train.corr()
# Mask the repeated values
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
  
f, ax = plt.subplots(figsize=(22, 18))
#Generate Heat Map, allow annotations and place floats in map
sns.heatmap(corr, annot=True, fmt=".2f" , mask=mask,)
    #Apply xticks
plt.xticks(range(len(corr.columns)), corr.columns);
    #Apply yticks
plt.yticks(range(len(corr.columns)), corr.columns)
    #show plot
plt.show()

In [0]:
data_train.drop(['TotalGasInNext6Months'],axis=1, inplace=True)

In [0]:
data_test.drop(['TotalGasInNext6Months'],axis=1, inplace=True)

In [0]:
data_test.drop(['WellID'] , axis=1, inplace=True )

In [0]:
data_train.drop(['WellID'] , axis=1, inplace=True )

In [0]:
data_train.drop(['Unnamed: 0'] , axis=1, inplace=True )

In [0]:
data_test.drop(['Unnamed: 0'] , axis=1, inplace=True )

In [0]:
data_train.drop(['firstMonthPercentWater','SecondMonthPercentWater','ThirdMonthPercentWater','FourthMonthPercentWater','FifthMonthPercentWater','SixthMonthPercentWater'],axis=1, inplace=True)

In [0]:
data_test.drop(['firstMonthPercentWater','SecondMonthPercentWater','ThirdMonthPercentWater','FourthMonthPercentWater','FifthMonthPercentWater','SixthMonthPercentWater'],axis=1, inplace=True)

In [0]:
data_test.drop(['FirstMonthDaysOn','SecondMonthDaysOn','ThirdMonthDaysOn','FourthMonthDaysOn','FifthMonthDaysOn','SixthMonthDaysOn'],axis=1, inplace=True)

In [0]:
data_train.drop(['FirstMonthDaysOn','SecondMonthDaysOn','ThirdMonthDaysOn','FourthMonthDaysOn','FifthMonthDaysOn','SixthMonthDaysOn'],axis=1, inplace=True)

In [0]:
data_train.drop(['SecondMonthWater','ThirdMonthWater','FourthMonthWater','FifthMonthWater','SixthMonthWater'],axis=1, inplace=True)

In [0]:
data_test.drop(['SecondMonthWater','ThirdMonthWater','FourthMonthWater','FifthMonthWater','SixthMonthWater'],axis=1, inplace=True)

In [0]:
# correlatio between fetures
corr = data_train.corr()
# Mask the repeated values
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
  
f, ax = plt.subplots(figsize=(22, 18))
#Generate Heat Map, allow annotations and place floats in map
sns.heatmap(corr, annot=True, fmt=".2f" , mask=mask,)
    #Apply xticks
plt.xticks(range(len(corr.columns)), corr.columns);
    #Apply yticks
plt.yticks(range(len(corr.columns)), corr.columns)
    #show plot
plt.show()

# selecting a model to train

In [0]:
seed = 79
np.random.seed(seed)

In [0]:
# Removing outliars
data_train_cleaned =  data_train[(np.abs(stats.zscore(data_train)) < 9).all(axis=1)]

In [0]:
len(data_train_cleaned)

In [0]:
len(data_train)

In [0]:
#spliting data to train/test
msk = np.random.rand(len(data_train_cleaned)) < 0.8

train = data_train_cleaned[msk]
test = data_train_cleaned[~msk]


In [0]:
train_X =  train.drop(['TotalOilInNext6Months'] , axis=1)
train_y = train['TotalOilInNext6Months']
test_X =  test.drop(['TotalOilInNext6Months'] , axis=1)
test_y = test['TotalOilInNext6Months']

In [0]:
models = [
           ['lin-regresion',LinearRegression(fit_intercept =True)],
           ['XGBRegressor: ', xgb.XGBRegressor()],
           ['MLPRegressor: ', MLPRegressor(  activation='relu', solver='adam',learning_rate='adaptive',max_iter=1400,learning_rate_init=0.02,alpha=0.01)] 
         ]

In [0]:
# Run all the proposed models and update the information in a list model_data
model_data = []
models_weights = []
for name,curr_model in models :
    curr_model_data = {}
    curr_model.random_state = 534
    curr_model_data["Name"] = name
    start = time.time()
    curr_model.fit(train_X,train_y)
    end = time.time()
    curr_model_data["Train_Time"] = end - start
    curr_model_data["Train_mean_absolute_error"] = (mean_absolute_error(train_y,curr_model.predict(train_X)))
    curr_model_data["Test_mean_absolute_error"] = (mean_absolute_error(test_y,curr_model.predict(test_X)))
    model_data.append(curr_model_data)

In [0]:
model_data

In [0]:
# Convert list to dataframe
scores = pd.DataFrame(model_data)

In [0]:
scores

# Gridsearch for best paremeters

In [0]:
xgb_model = xgb.XGBRegressor()


parameters = {'nthread':[4], 
              'objective':['reg:linear'],
              'learning_rate': [.03, 0.05, .07], 
              'max_depth': [5, 6, 7],
              'min_child_weight': [4],
              'silent': [1],
              'subsample': [0.7],
              'colsample_bytree': [0.7],
              'n_estimators': [500]}

clf = GridSearchCV(xgb_model, param_grid = parameters, cv = 5, n_jobs = -1 , scoring='neg_mean_absolute_error' , verbose=2)


In [0]:
clf.fit(train_X, train_y)

# convert results to csv to uploade it to kaggle

In [0]:
predections = clf.best_estimator_.predict(data_test)

In [0]:
lenn = len(predections)

In [0]:
np.reshape(predections, (lenn, 1))

In [0]:
dataid = pd.read_csv("/gdrive/My Drive/machathon-10-filteration-test/test.csv")
dataid['TotalOilInNext6Months'] = predections.tolist()
ready = dataid[['WellID','TotalOilInNext6Months']].copy()
ready.to_csv('/gdrive/My Drive/machathon-10-filteration-test/Submission.csv', index=False)